# PS7

## Laurence Warner 

In [117]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold

import statsmodels.api as sm

from sklearn.metrics import precision_score

from sklearn.ensemble import RandomForestClassifier

import math

from sklearn import svm

In [3]:
auto_data = pd.read_csv("Auto.csv", na_values = '?')  #convert to NaN

In [4]:
auto_data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
auto_data['mpg_binary'] = 0

In [6]:
med = np.median(auto_data.mpg)

In [7]:
for i in range(len(auto_data)):
    if auto_data.mpg.iloc[i] >= med:
        auto_data.mpg_binary.iloc[i] = 1

/Users/laurencewarner/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
quant = auto_data.drop(['mpg','name'], axis = 1)

In [9]:
quant_nonull = quant.dropna()

In [10]:
y = quant_nonull['mpg_binary']

In [11]:
quant_nonull = sm.add_constant(quant_nonull)
X = quant_nonull.drop('mpg_binary', axis = 1)

### 1

#### a 

In [62]:
X_vals = X.values
y_vals = y.values

In [63]:
kf_log = KFold(n_splits=4, shuffle=True, random_state=15)
kf_log

KFold(n_splits=4, random_state=15, shuffle=True)

In [64]:
MSE_vec = np.zeros(4)
prec0 = np.zeros(4)
prec1 = np.zeros(4)

In [65]:
def MSE_fold(y_test, y_pred):
    '''
    Sums squared errors for given fold.
    '''
    count = 0 
    for i in range(len(y_pred)):
        if y_pred[i] != y_test[i]:
            count += 1
    MSE = count/len(y_test)
    return MSE

Error rates = (1- precision).
precision = True Positive / All Positives

In [66]:
k_ind = 0

for train_index, test_index in kf_log.split(X):
    
    X_train, X_test = X_vals[train_index], X_vals[test_index]
    y_train, y_test = y_vals[train_index], y_vals[test_index]
    
    LR = LogisticRegression()
    LR.fit(X_train, y_train)
    y_pred = LR.predict(X_test)
    
    MSE_vec[k_ind] = MSE_fold(y_test, y_pred)
    prec0[k_ind] = (1 - precision_score(y_test, LR.predict(X_test), pos_label=0))
    prec1[k_ind] = (1 - precision_score(y_test, LR.predict(X_test), pos_label=1))
    
    k_ind += 1

In [67]:
MSE_vec

array([ 0.09183673,  0.10204082,  0.13265306,  0.10204082])

In [68]:
prec0

array([ 0.05769231,  0.12244898,  0.15217391,  0.04651163])

In [69]:
prec1

array([ 0.13043478,  0.08163265,  0.11538462,  0.14545455])

In [70]:
MSE = MSE_vec.mean()
print("Average Mean Squared Error = ", MSE)

Average Mean Squared Error =  0.107142857143


In [71]:
mean_prec0 = prec0.mean()
print("Mean error when predicting y_pred = 0 is:", mean_prec0)

Mean error when predicting y_pred = 0 is: 0.0947067070586


In [72]:
mean_prec1 = prec1.mean()
print("Mean error when predicting y_pred = 1 is:", mean_prec1)

Mean error when predicting y_pred = 1 is: 0.118226649127


#### b 

Random forest classifier, where features are chosen from those in part a

In [73]:
quant_nonull.columns

Index(['const', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'mpg_binary'],
      dtype='object')

In [74]:
len(X.columns)

8

In [75]:
math.sqrt(8)

2.8284271247461903

2 features slightly less than usual square root.

In [76]:
RFC = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)

X Vals with no constant

In [77]:
X_vals = quant_nonull.drop(['const', 'mpg_binary'], axis = 1).values

In [78]:
RFC.fit(X_vals, y_vals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [83]:
RFC.score(X_vals, y_vals)

1.0

Can't proceed: RandomForestClassifier has no oob_prediction_ attribute, only RandomForestRegressor.

#### c 

In [95]:
X_vals.shape

(392, 7)

In [97]:
y_vals.shape

(392,)

Firstly practise without using folds.

In [103]:
from sklearn.cross_validation import train_test_split

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X_vals, y_vals, random_state = 25)

In [120]:
s = svm.SVC(kernel='rbf', gamma=0.7)

In [121]:
s.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.7, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [122]:
y_pred = s.predict(X_test)

In [123]:
from sklearn.metrics import classification_report

In [127]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0])

In [128]:
y_test

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0])

In [126]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.45      1.00      0.62        43
          1       1.00      0.05      0.10        55

avg / total       0.76      0.47      0.33        98



Overpredicts category 0. Precision TP / (TP + FP) is very low. Lots of false positives.

In [144]:
s2 = svm.SVC(kernel='rbf', gamma=0.2)

In [145]:
s2.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [146]:
y_pred = s2.predict(X_test)

In [147]:
from sklearn.metrics import classification_report

In [148]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0])

In [149]:
y_test

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0])

In [150]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.45      1.00      0.62        43
          1       1.00      0.05      0.10        55

avg / total       0.76      0.47      0.33        98



Using gamma=0.2 hasn't changed anything!

In [143]:
svm.SVC?

In [98]:
kf_svm = KFold(n_splits=4, shuffle=True, random_state=15)
kf_svm?

In [152]:
MSE_vec = np.zeros(4)
prec0 = np.zeros(4)
prec1 = np.zeros(4)

In [155]:
X_vals

array([[   8. ,  307. ,  130. , ...,   12. ,   70. ,    1. ],
       [   8. ,  350. ,  165. , ...,   11.5,   70. ,    1. ],
       [   8. ,  318. ,  150. , ...,   11. ,   70. ,    1. ],
       ..., 
       [   4. ,  135. ,   84. , ...,   11.6,   82. ,    1. ],
       [   4. ,  120. ,   79. , ...,   18.6,   82. ,    1. ],
       [   4. ,  119. ,   82. , ...,   19.4,   82. ,    1. ]])

In [158]:
kf_svm.split?

In [167]:
k_ind = 0

for train_index, test_index in kf_svm.split(X_vals, y_vals):
    print("train_index:", train_index, "test_index:", test_index)
    X_train, X_test = X_vals[train_index], X_vals[test_index]
    y_train, y_test = y_vals[train_index], y_vals[test_index]
    
    s = svm.SVC(kernel='rbf', gamma=0.2)
    s.fit(X_train, y_train)
    y_pred = s.predict(X_test)
    
    MSE_vec[k_ind] = MSE_fold(y_test, y_pred)
    prec0[k_ind] = (1 - precision_score(y_test, y_pred, pos_label=0))
    prec1[k_ind] = (1 - precision_score(y_test, y_pred, pos_label=1))
    
    k_ind += 1

train_index: [  0   1   3   4   5   7   8  10  11  12  14  15  16  17  19  20  22  23
  26  27  28  29  30  31  32  34  36  37  39  40  41  42  43  45  46  48
  49  50  51  52  53  55  56  57  59  60  61  62  63  66  68  69  70  71
  72  74  75  76  77  78  79  81  82  84  85  87  88  89  90  91  92  93
  94  95  96  98  99 100 101 102 104 107 108 109 112 113 114 115 117 118
 119 120 121 122 123 128 130 132 133 134 136 137 139 140 141 142 143 144
 145 146 147 148 150 151 152 153 154 155 156 157 159 160 161 162 163 165
 166 167 168 169 171 173 175 176 177 179 180 181 182 183 184 186 187 188
 189 193 195 196 198 199 201 202 203 205 206 207 208 210 211 212 213 215
 216 217 218 219 220 221 223 224 225 226 227 229 230 231 232 234 235 236
 237 238 240 241 243 244 245 246 247 249 250 251 252 253 255 256 259 260
 262 263 265 266 267 270 271 274 278 279 280 281 282 283 284 289 291 292
 293 294 295 296 298 299 300 301 302 303 304 305 306 307 308 309 310 311
 312 313 314 316 317 318 319 322 324 3

/Users/laurencewarner/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [168]:
MSE_vec

array([ 0.54081633,  0.52040816,  0.52040816,  0.44897959])

In [169]:
prec0

array([ 0.        ,  0.52040816,  0.53125   ,  0.47311828])

In [170]:
prec1

array([ 0.55208333,  1.        ,  0.        ,  0.        ])

In [171]:
MSE = MSE_vec.mean()
print("Average Mean Squared Error = ", MSE)

Average Mean Squared Error =  0.507653061224


In [172]:
mean_prec0 = prec0.mean()
print("Mean error when predicting y_pred = 0 is:", mean_prec0)

Mean error when predicting y_pred = 0 is: 0.381194110709


In [174]:
mean_prec1 = prec1.mean()
print("Mean error when predicting y_pred = 1 is:", mean_prec1)

Mean error when predicting y_pred = 1 is: 0.388020833333


#### d 

I can only compare Logistic Regression and Support Vector Machine.

Logistic regression has an average MSE of around 0.1, whereas the SVM has an average MSE of around 0.5 (which means it is not better than random).

This implies that Logistic Regression is more suitable, although there could be a problem with my SVM implementation. 